In [ ]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration
import pandas as pd
import os

tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base')

for path, subdirs, files in os.walk("/Users/brendankondracki/Documents/CodeContests/code-contests-csv/"):
    for filename in files:
        print(filename)
        if ".csv" in filename and "clean_" not in filename:
            df = pd.read_csv(os.path.join(path, filename))
            valid_data = []
            long_questions = set()
            short_questions = set()
            for index, row in df.iterrows():
                if row["Solution Language"] not in ("PYTHON", "PYTHON3", "JAVA", "CPP"):
                    valid_data.append(0)
                elif row["Description"] in long_questions:
                    valid_data.append(0)
                else:
                    if row["Description"] not in short_questions:
                        input_text = str(row["Description"]) + "</s>"
                        input_ids = tokenizer(input_text, return_tensors="pt").input_ids
                        if len(input_ids[0]) > 507:
                            long_questions.add(row["Description"])
                            valid_data.append(0)
                            continue
                        else:
                            short_questions.add(row["Description"])
                            
                    if pd.isnull(row["Incorrect Solution"]):
                        input_text = str(row["Description"]) + "</s>" + str(row["Correct Solution"])
                        input_ids = tokenizer(input_text, return_tensors="pt").input_ids
                        if len(input_ids[0]) > 507:
                            valid_data.append(0)
                        else:
                            valid_data.append(1)
                    else:
                        input_text = row["Description"] + "</s>" + row["Incorrect Solution"]
                        input_ids = tokenizer(input_text, return_tensors="pt").input_ids
                        if len(input_ids[0]) > 507:
                            valid_data.append(0)
                        else:
                            valid_data.append(1)

            df["valid_data"] = valid_data
            df = df[df["valid_data"] == 1]
            df.pop("valid_data")
            df.to_csv(os.path.join(path, "clean_" + filename), index=False)